In [59]:
from imutils import face_utils
import numpy as np 
import imutils 
import dlib 
import cv2 

In [60]:
img_path = 'test.jpg' #합성할 이미지

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("landmark/shape_predictor_68_face_landmarks.dat")

    
mask_img = cv2.imread('mask.jpg')
image = cv2.imread(img_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
rects = detector(gray, 1)

#일단 이미지에서 사람 한명만 찾는걸로 구현

for (i,rect) in enumerate(rects):
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    
left = np.array(shape[1],dtype=np.int64)
right = np.array(shape[15],dtype=np.int64)
down = np.array(shape[8],dtype=np.int64) 
k = np.vstack([down,np.vstack([left,right])])

(x,y,w,h) = cv2.boundingRect(np.array([k]))
    
mask_area =  image[y:y+h,x:x+w]
ma_w,ma_h,ma_c = mask_area.shape
mask_img = cv2.resize(mask_img,(ma_h,ma_w))

maskgray =cv2.cvtColor(mask_img,cv2.COLOR_BGR2GRAY)
ret,masked = cv2.threshold(maskgray,100,255,cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(masked)
img_fg = cv2.bitwise_and(mask_img,mask_img,mask=masked)
img_bg = cv2.bitwise_and(mask_area,mask_area,mask=mask_inv)
result = cv2.add(img_fg,img_bg)

image[y:y+h,x:x+w] = result

cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('mask_{}'.format(img_path),image)

True